In [8]:
from mistralai.async_client import MistralAsyncClient
from mistralai.models.chat_completion import ChatMessage

api_key = MISTRAL_API_KEY
model = "mistral-tiny"

client = MistralAsyncClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]

# With async
async_response = client.chat(model=model, messages=messages)

async for chunk in async_response: 
    print(chunk.choices[0].delta.content)

TypeError: 'async for' requires an object with __aiter__ method, got coroutine

In [66]:
import time
import json
from mistralai.client import MistralClient
from mistralai.async_client import MistralAsyncClient

from mistralai.models.chat_completion import ChatMessage
from pydantic import BaseModel, ValidationError, ConfigDict
from typing import Type, Optional
import os
from dotenv import load_dotenv
import asyncio
import nest_asyncio
nest_asyncio.apply()

load_dotenv()
MISTRAL_API_KEY = "GFBjsGogmbv0LuMWjJewXBXwyN7QeKNj"


class MistralLanguageModel:
    def __init__(self, api_key=MISTRAL_API_KEY,
                 model="mistral-tiny", temperature=0.0):

        if api_key is None:
            raise ValueError("The Mistral API KEY must be provided either as "
                             "an argument or as an environment variable named 'MISTRAL_API_KEY'") # noqa

        self.api_key = api_key
        self.model = model
        self.temperature = temperature
        
        #self.client = MistralAsyncClient(api_key=self.api_key)
        self.client = MistralClient(api_key=self.api_key)

    def async_generation(self,  prompt: str, #async
                 output_format: Optional[Type[BaseModel]] = None,
                 max_tokens: int = None):
        system_message = "You are a helpful assistant."
        if output_format:
            system_message += f" Respond in a JSON format that contains the following keys: {self._model_structure_repr(output_format)}. You must only return a JSON, nothing else. You are strictly forbidden to return anything else, like an explanation." # noqa

        params = {
            "model": self.model,
            "messages": [
                ChatMessage(role="system", content=system_message),
                ChatMessage(role="user", content=prompt)
            ],
            "temperature": self.temperature}

        if max_tokens is not None:
            params["max_tokens"] = max_tokens

        return self.client.chat(**params)
    

    def generate(self, prompts: list[str],
                 output_format: Optional[Type[BaseModel]] = None,
                 max_tokens: int = None):

        

        """loop = asyncio.get_event_loop()
        tasks = [loop.create_task(self.async_generation(prompt, output_format)) for prompt in prompts]

        responses = loop.run_until_complete(asyncio.gather(*tasks))
        responses = [self.async_generation(prompt, output_format) for prompt in prompts]
        """
        responses = []
        for prompt in prompts:
            res = self.async_generation(prompt, output_format).choices[0].message.content
            print(res)
            assert self._is_valid_json_for_model(res, output_format)
            responses.append(res)
        #responses = [response.choices[0].message.content for response in responses]
        return responses
        
        print(responses)
        if output_format:
            for res in responses:
                assert self._is_valid_json_for_model(res, output_format)
                
        return responses

    def _model_structure_repr(self, model: Type[BaseModel]) -> str:
        fields = model.__annotations__
        return ', '.join(f'{key}: {value}' for key, value in fields.items())

    def _is_valid_json_for_model(self, text: str, model: Type[BaseModel]) -> bool: # noqa
        """
        Check if a text is valid JSON and if it respects the provided BaseModel. # noqa
        """
        model.model_config = ConfigDict(strict=True)

        try:
            parsed_data = json.loads(text)
            model(**parsed_data)
            return True
        except (json.JSONDecodeError, ValidationError):
            parsed_data = json.loads(text)
            model(**parsed_data)
            return False


class Output(BaseModel):
    first_name: str
    last_name: str
    city: str


llm = MistralLanguageModel()
prompts = [
    'Extract the requested  information from the following sentence: "Alice Johnson is visiting Rome."',
    'Extract the requested  information from the following sentence: "Emmanuel Macron is visiting New York."',
    'Extract the requested  information from the following sentence: "Growing Ananas is visiting Paris."'
]
response = llm.generate(prompts, output_format=Output)

print(response)

{
  "first_name": "Alice",
  "last_name": "Johnson",
  "city": "Rome"
}
{
  "first_name": "Emmanuel",
  "last_name": "Macron",
  "city": "New York"
}
{
  "first_name": "Growing",
  "last_name": "Ananas",
  "city": "Paris"
}
['{\n  "first_name": "Alice",\n  "last_name": "Johnson",\n  "city": "Rome"\n}', '{\n  "first_name": "Emmanuel",\n  "last_name": "Macron",\n  "city": "New York"\n}', '{\n  "first_name": "Growing",\n  "last_name": "Ananas",\n  "city": "Paris"\n}']


In [57]:
class MistralLanguageModel:
    def __init__(self, api_key=MISTRAL_API_KEY,
                 model="mistral-tiny", temperature=0.5):

        if api_key is None:
            raise ValueError("The Mistral API KEY must be provided either as "
                             "an argument or as an environment variable named 'MISTRAL_API_KEY'") # noqa

        self.api_key = api_key
        self.model = model
        self.temperature = temperature
        self.client = MistralClient(api_key=self.api_key)

    def generate(self, prompt: str,
                 output_format: Optional[Type[BaseModel]] = None,
                 max_tokens: int = None):

        retry_delay = 0.1

        while True:
            if True:
            #try:
                system_message = "You are a helpful assistant."
                if output_format:
                    system_message += f" Respond in a JSON format that contains the following keys: {self._model_structure_repr(output_format)}. You must only return a JSON, nothing else. You are strictly forbidden to return anything else, like an explanation." # noqa

                messages = [
                    ChatMessage(role="system", content=system_message),
                    ChatMessage(role="user", content=prompt)
                ]
                params = {
                    "model": self.model,
                    "messages": messages,
                    "temperature": self.temperature
                }

                if max_tokens is not None:
                    params["max_tokens"] = max_tokens

                response = self.client.chat(**params)
                response_content = response.choices[0].message.content
                print(response_content)

                if output_format:
                    assert self._is_valid_json_for_model(response_content,
                                                     output_format)
                    return response_content
                #else:
                #    return response_content

            #except Exception:
            #    print(f"Hit rate limit. Retrying in {retry_delay} seconds.")
            #    time.sleep(retry_delay)
            #    retry_delay *= 2

    def _model_structure_repr(self, model: Type[BaseModel]) -> str:
        fields = model.__annotations__
        return ', '.join(f'{key}: {value}' for key, value in fields.items())


    def _is_valid_json_for_model(self, text: str, model: Type[BaseModel]) -> bool: # noqa
        """
        Check if a text is valid JSON and if it respects the provided BaseModel. # noqa
        """
        model.model_config = ConfigDict(strict=True)

        try:
            parsed_data = json.loads(text)
            model(**parsed_data)
            return True
        except (json.JSONDecodeError, ValidationError):
            return False
llm = MistralLanguageModel()


In [61]:
prompts = [
    'Extract the requested  information from the following sentence: "Alice Johnson is visiting Rome."',
    'Extract the requested  information from the following sentence: "Emmanuel Macron is visiting New York."'
    'Extract the requested  information from the following sentence: "Growing Ananas is visiting Paris."'
]
responses = []
from time import sleep
for prompt in prompts:
    responses.append(llm.generate(prompt, output_format=Output))
    sleep(1)

{
  "first_name": "Alice",
  "last_name": "Johnson",
  "city": "Rome"
}
{
  "first_name": "Emmanuel",
  "last_name": "Macron",
  "city": "New York"
}
{
  "first_name": "Growing",
  "last_name": "Ananas",
  "city": "Paris"
}


AssertionError: 

In [38]:
json.loads("""{
  "first_name": "Growing",
  "last_name": "Ananas",
  "city": "Paris"
}""")

{'first_name': 'Growing', 'last_name': 'Ananas', 'city': 'Paris'}

In [32]:
l = ['{\n  "first_name": "Alice",\n  "last_name": "Johnson",\n  "city": "Rome"\n}', '{\n  "first_name": "Emmanuel",\n  "last_name": "Macron",\n  "city": "New York"\n}\n','\n{\n  "first_name": "Growing",  "last_name": "Ananas",\n  "city": "Paris"\n}']


eval(l[1])

{'first_name': 'Emmanuel', 'last_name': 'Macron', 'city': 'New York'}

In [6]:
pip install openai==0.28

  Using cached openai-0.28.0-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.10.0
    Uninstalling openai-1.10.0:
      Successfully uninstalled openai-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
instructor 0.4.8 requires openai<2.0.0,>=1.1.0, but you have openai 0.28.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openai==1.10.0

  Using cached openai-1.10.0-py3-none-any.whl.metadata (18 kB)
Using cached openai-1.10.0-py3-none-any.whl (225 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://vigieinstance.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "6e612f025340400d827a519b0549cff6"

message_text = [{"role":"system","content":"Assistant is an AI chatbot that helps users turn a natural language list into JSON format. After users input a list they want in JSON format, it will provide suggested list of attribute labels if the user has not provided any, then ask the user to confirm them before creating the list."}]

completion = openai.ChatCompletion.create(
  engine="dep",
  messages = message_text,
  temperature=0.2,
  max_tokens=350,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [2]:
completion

<OpenAIObject chat.completion id=chatcmpl-8s0I2aZv3YMhrHcciMxIAiBAG80qd at 0x10a456a50> JSON: {
  "id": "chatcmpl-8s0I2aZv3YMhrHcciMxIAiBAG80qd",
  "object": "chat.completion",
  "created": 1707881518,
  "model": "gpt-4",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {}
    }
  ],
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Sure, I can help you with that. Please provide me with the natural language list that you would like to convert into JSON format. If you have specific attribute labels in mind, include them as well; otherwise, I can suggest some for you."
      },
      "content_filter_result": {
        "error": {
          "code": "content_filter_error",
          "message": "The contents are not filtered"
        }
      },
      "content_filter_results": {}
    }
  ],
  "usage": {
    "prompt_tokens": 64,
    "completion_tokens": 50,
    "to

In [ ]:
#Note: The openai-python library support for Azure OpenAI is in preview.
      #Note: This code sample requires OpenAI Python library version 0.28.1 or lower.
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://vigieinstance.openai.azure.com/"
#openai.api_version = "2023-07-01-preview"
openai.api_key = "6e612f025340400d827a519b0549cff6"

from openai import OpenAI, AsyncOpenAI
client = OpenAI()

message_text = [
    #{"role":"system","content":"Assistant is an AI chatbot that helps users turn a natural language list into JSON format. After users input a list they want in JSON format, it will provide suggested list of attribute labels if the user has not provided any, then ask the user to confirm them before creating the list."},
    {"role": "user", "content": "Say this is a test"},
    ]

completion = client.chat.completions.create(
  #engine="dep",
  model="gpt-4",
  messages = message_text,
  temperature=0.2,
  max_tokens=350,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)